In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import root_mean_squared_error

In [2]:
import mlflow
mlflow.set_tracking_uri("sqlite:///../mlflow.db")
mlflow.set_experiment("new-experiment")


<Experiment: artifact_location='/workspaces/mlops/02-tracking/mlruns/1', creation_time=1716817360881, experiment_id='1', last_update_time=1716817360881, lifecycle_stage='active', name='new-experiment', tags={}>

### Train Linear Regression

In [3]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [4]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet') #pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [5]:
len(df_train), len(df_val)

(73908, 61921)

In [6]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [7]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer(sparse=True)

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [9]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715209663881

In [14]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

### MLFlow experiment tracking

In [20]:
with mlflow.start_run():
    mlflow.set_tag("developer", "Azamat")
    mlflow.log_param("train-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")
    alpha = 1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha = alpha)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

### Hyperparameter Tuning

In [10]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [11]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [12]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [26]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)   

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/envs/exp_tracking/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:15:33] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.41217                          
[1]	validation-rmse:9.16431                           
[2]	validation-rmse:8.31864                           
[3]	validation-rmse:7.75804                           
[4]	validation-rmse:7.39120                           
[5]	validation-rmse:7.15093                           
[6]	validation-rmse:6.99549                           
[7]	validation-rmse:6.89241                           
[8]	validation-rmse:6.82111                           
[9]	validation-rmse:6.77364                           
[10]	validation-rmse:6.73992                          
[11]	validation-rmse:6.71637                          
[12]	validation-rmse:6.69729                          
[13]	validation-rmse:6.68334                          
[14]	validation-rmse:6.67394                          
[15]	validation-rmse:6.66759                          
[16]	validation-rmse:6.66113                          
[17]	validation-rmse:6.65709                          
[18]	valid

/home/codespace/anaconda3/envs/exp_tracking/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:16:59] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.18560                                                     
[1]	validation-rmse:8.83484                                                      
[2]	validation-rmse:7.96027                                                      
[3]	validation-rmse:7.40864                                                      
[4]	validation-rmse:7.06622                                                      
[5]	validation-rmse:6.85501                                                      
[6]	validation-rmse:6.71999                                                      
[7]	validation-rmse:6.62960                                                      
[8]	validation-rmse:6.57134                                                      
[9]	validation-rmse:6.52772                                                      
[10]	validation-rmse:6.49911                                                     
[11]	validation-rmse:6.47796                                                     
[12]	validation-

...

/home/codespace/anaconda3/envs/exp_tracking/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:26:49] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.05367                                                       
[1]	validation-rmse:7.61437                                                       
[2]	validation-rmse:7.00509                                                       
[3]	validation-rmse:6.74482                                                       
[4]	validation-rmse:6.62411                                                       
[5]	validation-rmse:6.56460                                                       
[6]	validation-rmse:6.52994                                                       
[7]	validation-rmse:6.50798                                                       
[8]	validation-rmse:6.49583                                                       
[9]	validation-rmse:6.48662                                                       
[10]	validation-rmse:6.48203                                                      
[11]	validation-rmse:6.47641                                                      
[12]

/home/codespace/anaconda3/envs/exp_tracking/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:27:48] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.56127                                                      
[1]	validation-rmse:9.37504                                                       
[2]	validation-rmse:8.54059                                                       
[3]	validation-rmse:7.96502                                                       
[4]	validation-rmse:7.57081                                                       
[5]	validation-rmse:7.30428                                                       
[6]	validation-rmse:7.12394                                                       
[7]	validation-rmse:7.00195                                                       
[8]	validation-rmse:6.91681                                                       
[9]	validation-rmse:6.85543                                                       
[10]	validation-rmse:6.81118                                                      
[11]	validation-rmse:6.78089                                                      
[12]

KeyboardInterrupt: 

In [13]:
mlflow.autolog()

with mlflow.start_run():

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)

2024/05/27 14:38:31 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


2024/05/27 14:38:33 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


: 